# Neural Amp Modeler ("Easy Mode" Trainer)
**Note**:
This notebook is meant to be used on [Google Colab](https://colab.research.google.com/github/sdatkinson/neural-amp-modeler/blob/main/bin/train/easy_colab.ipynb).

🔶**Before you run**🔶

Make sure to get a GPU! (From the upper-left menu, click Runtime->Change runtime type->Select "GPU" from the "Hardware accelerator dropdown menu)

## Step 1: Get data
* **Download the reamp signal.** Here: [v3_0_0.wav](https://drive.google.com/file/d/1Pgf8PdE0rKB1TD4TRPKbpNo1ByR3IOm9/view?usp=drive_link).
* **Reamp your gear.** Then reamp the gear you want to model using it. Save that reamp as "output.wav". *Note: Use 48kHz, 24-bit, mono.* For other sample rates, use [the CLI trainer](https://github.com/sdatkinson/neural-amp-modeler).
* **Upload your files.** Upload the input (DI) and output (amped) files you want to use by clicking the Folder icon on the left ⬅ and then clicking the upload icon or by dragging the files into the panel.

## Step 2: Train!
Configure your training run below, then hit the Play button to start training!

🕙NOTE: At default settings, training will take just over 10 minutes.🕙

In [ ]:
try:
    import nam
except ImportError as e:
    !pip install neural-amp-modeler
    # Hint: use the next line instead for the very latest!
    # !pip install git+https://github.com/sdatkinson/neural-amp-modeler.git@main

from nam.train.colab import run
from nam.models.metadata import GearType, ToneType, UserMetadata

%load_ext tensorboard

from functools import partial

import ipywidgets as widgets

# NOTE: Enums need to be handled carefully since the values need to be supplied literally here!

#@markdown # Training parameters
epochs = 100 #@param {type: "number"}
architecture = "standard"  #@param ["standard", "lite", "feather", "nano"] {type: "string"}
fit_cab = False  #@param {type: "boolean"}
ignore_checks = False #@param {type: "boolean"}

#@markdown # Metadata
use_metadata = False #@param {type: "boolean"}
name = "My model" #@param {type:"string"}
modeled_by = "Your name" #@param {type:"string"}
gear_make = "GearCo" #@param {type:"string"}
gear_model = "GearName" #@param {type:"string"}
gear_type = "pedal" #@param ["amp", "pedal", "pedal_amp", "amp_cab", "amp_pedal_cab", "preamp", "studio"] {type:"string"}
tone_type = "overdrive" #@param ["clean", "overdrive", "crunch", "hi_gain", "fuzz"] {type:"string"}

def _verbose_enum(E, val):
    try:
        return E(val)
    except ValueError as e:
        raise ValueError(
            str(e)
            + "\nValid choices are: "
            + ", ".join(list(x.value for x in E))
        )

user_metadata = None if not use_metadata else UserMetadata(
    name=name,
    modeled_by=modeled_by,
    gear_make=gear_make,
    gear_model=gear_model,
    gear_type=_verbose_enum(GearType, gear_type.lower()),
    tone_type=_verbose_enum(ToneType, tone_type.lower())
)
run_partial = partial(run, user_metadata=user_metadata)

%tensorboard --logdir /content/lightning_logs
run(
    epochs=epochs,
    architecture=architecture,
    fit_cab=fit_cab,  # Change me to True for full-rig modeling!
    ignore_checks=ignore_checks,  # Change to True to train anyways with potentially bad data
    user_metadata=user_metadata
)

## Step 3: Check the results and download your model
We're done!

Have a look at the plot above to see how your model compares to the real gear you're modeling.
Hopefully it looks good!
Go to the file browser on the left panel ⬅ and download `model.nam` from the `exported_model` directory (you may need to hit the refresh button).

Additionally, if you want to continue to train this model later you can download the lightning model artifacts from `lightning_logs`. If not, that's fine too.

# 🎸 **ENJOY!** 🎸